# Deep Q-Network

# Import

In [1]:
import gym
import random
import numpy as np
from collections import deque
from time import sleep
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Reshape
from IPython.display import clear_output

# Environment

In [2]:
env = gym.make('Taxi-v3').env

In [4]:
env.action_space

Discrete(6)

In [5]:
env.observation_space

Discrete(500)

# Model

In [6]:
class DQN(Sequential):
    def __init__(self, n_embedded=10, n_nodes=50, n_hidden=2):
        super().__init__()
        # input layer
        self.add(Embedding(env.observation_space.n, n_embedded, input_length=1))
        self.add(Reshape((n_embedded, )))
        # hidden layers
        for _ in range(n_hidden):
            self.add(Dense(n_nodes, activation='relu'), )
        # output layer
        self.add(Dense(env.action_space.n, activation='linear'),)
        # compile
        self.compile(loss='mse', optimizer='adam')

    def clone_from(self, another):
        self.set_weights(another.get_weights())
        return self

In [7]:
policy = DQN()

In [8]:
policy.summary()

Model: "dqn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             5000      
_________________________________________________________________
reshape (Reshape)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                550       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total params: 8,406
Trainable params: 8,406
Non-trainable params: 0
_________________________________________________________________


# Replay Memory

In [9]:
class Memory:
    def __init__(self, maxlen):
        self._memory = deque(maxlen=maxlen)
    
    def remember(self, state, action, reward, next_state, done):
        self._memory.append((state, action, reward, next_state, done))
    
    def get_batch(self, batch_size):
        samples = random.sample(self._memory, min(len(self._memory), batch_size))
        batch = np.array(samples, dtype=object).transpose()
        states, actions, rewards, next_states, dones = batch
        states, next_states = np.stack(states), np.stack(next_states)
        return states, actions, rewards, next_states, dones

# Agent

* Initialize replay memory capacity.
* Initialize the policy network with random weights.
* Clone the policy network, and call it the target network.
* For each episode:
    * Initialize the starting state.
    * For each time step:
        * Select an action.
            * Via exploration or exploitation
        * Execute selected action in an emulator.
        * Observe reward and next state.
        * Store experience in replay memory.
        * Sample random batch from replay memory.
        * Preprocess states from batch.
        * Pass batch of preprocessed states to policy network.
        * Calculate loss between output Q-values and target Q-values.
            * Requires a pass to the target network for the next state
        * Gradient descent updates weights in the policy network to minimize loss.
            * After time steps, weights in the target network are updated to the weights in the policy network.

In [10]:
class Agent:
    def __init__(self, env, policy):
        self._env = env
        self._memory = Memory(100000)
        self._policy = policy
        self._target = DQN().clone_from(self._policy)

    @property
    def policy(self): return self._policy
    
    def choose_action(self, state, epsilon=1.0):
        if np.random.uniform(0,1)<=epsilon:
            action = self._env.action_space.sample()            
        else:            
            action = np.argmax(self._policy(tf.constant([state])))
        return action

In [11]:
class Agent(Agent):
    def play(self, render=False, n_steps=200):
        state = env.reset()
        done = False
        rewards = 0
        for i_steps in range(1,n_steps+1):
            action = np.argmax(self._policy(tf.constant([state])))
            next_state, reward, done, info = env.step(action)
            rewards += reward
            if render: 
                clear_output(wait=True)
                env.render()
                sleep(0.05)
            if done: 
                break
            state = next_state
        if render: 
            print(f'Steps taken: {steps}, rewards earned: {rewards}')
            env.close()
        else:
            return rewards

In [12]:
class Agent(Agent):    
    def train(self, batch_size=512, gamma=0.99):
        states, actions, rewards, next_states, dones = self._memory.get_batch(batch_size)
        next_states_predicts = self._target(next_states).numpy()
        q_targets = self._policy(states).numpy()        
        for i,row in enumerate(q_targets):
            row[actions[i]] = rewards[i] if dones[i] else rewards[i] + gamma*np.max(next_states_predicts[i])
        X, y = tf.constant(states), tf.constant(q_targets)
        self._policy.fit(X, y, epochs=10, batch_size=batch_size, verbose=0)

In [13]:
class Agent(Agent):
    def run(self, n_eps=5001, max_steps=200):
        scores = deque(maxlen=100)
        for i_eps in range(1, n_eps+1):            
            state = env.reset()
            done = False            
            for _ in range(max_steps):
                action = self.choose_action(state, epsilon=i_eps/n_eps)
                next_state, reward, done, info = env.step(action)
                self._memory.remember(state, action, reward, next_state, done)
                state = next_state
                if done: break
            self.train()
            if i_eps%10==0:
                scores.append(self.play(render=False))
            if i_eps%50==0:
                self._target.clone_from(self._policy)                
            if i_eps%2==0:
                print('#', end='')
                if i_eps%100==0:               
                    avg_score = sum(scores)/len(scores)
                    print(f' | Episode {i_eps:>4d} | rewards: {avg_score:.1f}', end='\n')
                    if avg_score>12:
                        break

In [14]:
agent = Agent(env, policy)

In [15]:
agent.run()

################################################## | Episode  100 | rewards: -200.0
################################################## | Episode  200 | rewards: -200.0
################################################## | Episode  300 | rewards: -200.0
################################################## | Episode  400 | rewards: -244.1
################################################## | Episode  500 | rewards: -235.3
################################################## | Episode  600 | rewards: -259.4
################################################## | Episode  700 | rewards: -250.9
################################################## | Episode  800 | rewards: -244.6
################################################## | Episode  900 | rewards: -239.6
################################################## | Episode 1000 | rewards: -235.6
################################################## | Episode 1100 | rewards: -235.6
################################################## | Episode 1200 | rewards:

# Evaluation

In [20]:
agent.play(render=True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Steps taken: 11, rewards earned: 10


In [17]:
if input('Save model? ([Y]/n)').upper()=='Y':
    policy.save('taxi_v3_DQN.h5')

Save model? ([Y]/n) y


# Comment

* DQN may be unnecessary for this finite environment, but I tried anyway.
* Training time is slower than using a Q-Table, always use the simplest available model.
* The replay memory may have increased the number of training samples and therefore increased the training time, but it is necessary to relearn from past experience for more complex environment.
* Most solution on the Internet suggests to use two separated networks for policy and target, and only clone the weights of the policy to the target after certain steps to keep the Q-value target stable.
* The principal of Q-Learning and DQN are the same, to approximate the `State-Value function` or `Q-function` by exploring the environment and observing the `temporal difference` between two states. If the approximation is good enough, the temporal difference should be minimal and thus satisfying the [`Bellman equation`](https://en.wikipedia.org/wiki/Bellman_equation).